In [1]:
from tqdm import tqdm
import datetime as dt
import pymssql
import pandas as pd
import numpy as np
import warnings
import glob 
import os

warnings.filterwarnings('ignore')

In [2]:
conn = pymssql.connect(server='AX-SQL', database='Staging')
cursor = conn.cursor()

sql = f'''
SELECT [ID Сайта], [Артикул поставщика], [Товарная группа], [Товарное направление], [Название товара], [Код склада], [Остаток], [Код товара]
from [Staging].[Reports].[Remainings_8h]
where [Товарное направление] in (N'ЦБ', N'СИ')
'''

sql_query = pd.read_sql_query(sql, conn)
stock = pd.DataFrame(sql_query)

In [3]:
path = r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Архив_отчеты\Продажи\По дням\\'
list_of_files = glob.glob(path + '*.xlsx') 
latest_file = max(list_of_files, key=os.path.getctime)

sklad = pd.read_excel(latest_file, sheet_name = 'Продажи по ассортименту', header = 2)

stock['Код склада'] = stock['Код склада'].astype('float64') 

sklad = sklad[['Unnamed: 0','Торговая марка', 'Юр лицо', 'Код склада', 'Дата открытия объекта', 'Код объекта', 'Город', 'Наименование объекта']]
sklad1 = sklad[['Код склада', 'Код объекта']].drop_duplicates().reset_index(drop=True)
sklad1 = sklad1[sklad1['Код объекта'].notna()]

sklad1['Код объекта'] = sklad1['Код объекта'].astype(str)
sklad1['8'] = sklad1['Код объекта'].apply(lambda x: x[0])
sklad1 = sklad1[sklad1['8'] != '8']

stock = stock.merge(sklad1, left_on='Код склада', right_on = 'Код склада', how='inner')

stock = stock.drop(columns=['8'])

In [4]:
stock = stock[stock['ID Сайта'] != '']

In [5]:
ost = stock.groupby(by = 'ID Сайта', as_index = False)['Остаток'].sum()

In [6]:
stock.drop(columns=['Остаток', 'Код объекта', 'Код склада'], inplace=True)

In [7]:
os.chdir(r'C:\\Users\Trenkin.Sergey\Desktop\ecomsales')
extension = 'xlsx'
all_sales = [i for i in glob.glob('*.{}'.format(extension))]

sale = pd.DataFrame()
for name in tqdm(all_sales):
    sales = pd.read_excel(name)
    sale = pd.concat([sales, sale], ignore_index= True)

100%|██████████| 16/16 [02:45<00:00, 10.34s/it]


In [8]:
sale = sale.reset_index(drop=True)
sale.drop_duplicates(inplace=True)

sale = sale[(sale['state_order'] != 'returned') & (sale['state_order'] != 'canceled')]
sale = sale[~((sale['delivery_type'] == 'PICKUP_NOW') & (sale['payment_type'] == 'CASHIER') & (sale['type_ibox'] == 'CASHIER'))|
                ((sale['delivery_type'] == 'PICKUP') & (sale['payment_type'] == 'CASHIER') & (sale['type_ibox'] == 'CASHIER'))]

In [9]:
sale = sale[~sale['client_type'].str.contains('OZON')]

In [10]:
sale['result'] = sale['price_wo_manzana']
sale['result'].fillna(sale['price'], inplace=True)

In [11]:
def str_float(row):
    row = str(row)
    row = row.replace(',','.')
    return row

sale['price'] = sale['price'].apply(str_float)
sale['price'] = sale['price'].astype(float)
sale = sale[sale['price'] > 1].reset_index(drop=True)

In [12]:
sebes = sale.groupby(by = 'tov', as_index=False)['cost_price'].sum()

In [13]:
online = sale.groupby(by = 'tov', as_index=False)['result'].agg({'sum', 'count'})

In [14]:
online = online.merge(sebes, how='left', on='tov')

In [15]:
online.rename(columns={'tov': 'Код товара', 'sum': 'Сумма продаж', 'count': 'Количество продаж', 'cost_price': 'Себестоимость'}, inplace=True)

In [16]:
online

,Код товара,Сумма продаж,Количество продаж,Себестоимость
0,ТОВ0115302,33764.000000,1,15031.88
1,ТОВ0115418,4292.100000,1,1315.80
2,ТОВ0120217,8424.000000,1,3680.03
3,ТОВ0120218,8977.500000,1,4004.34
4,ТОВ0120219,16906.200000,1,4278.07
...,...,...,...,...
82689,ТОВ1329932,2450.000000,1,954.99
82690,ТОВ1329934,5480.000000,1,2355.00
82691,ТОВ1329935,5480.000000,1,2355.00
82692,ТОВ1330947,2666.689557,1,1820.48


In [17]:
sale = pd.read_excel(r'C:\Users\Trenkin.Sergey\Desktop\Продажи по ТОВ СИ, ЦБ янв.23-июн.24.xlsx')

In [18]:
sale.rename(columns={'Номер': 'Код товара'}, inplace=True)

In [19]:
sale = sale.merge(online, how='left', on='Код товара')

In [20]:
sale

,Код товара,Количество-продажи,Сумма продаж с баллами,Себестоимость продаж,Сумма продаж,Количество продаж,Себестоимость
0,ТОВ0115302,5,198154.16,65596.85,33764.0,1.0,15031.88
1,ТОВ0115418,1,4292.10,1315.80,4292.1,1.0,1315.80
2,ТОВ0115554,1,66654.28,15108.45,NaN,NaN,NaN
3,ТОВ0119984,1,11000.00,2060.18,NaN,NaN,NaN
4,ТОВ0120217,1,8424.00,3680.03,8424.0,1.0,3680.03
...,...,...,...,...,...,...,...
104956,ТОВ1335466,2,10449.30,7512.00,NaN,NaN,NaN
104957,ТОВ1335467,4,23400.00,15412.00,NaN,NaN,NaN
104958,ТОВ1335468,1,6060.00,3986.00,NaN,NaN,NaN
104959,ТОВ1335469,2,12920.00,8498.00,NaN,NaN,NaN


In [21]:
sale.fillna(0, inplace=True)

In [22]:
sale['Сумма'] = sale['Сумма продаж'] + sale['Сумма продаж с баллами']
sale['Себес'] = sale['Себестоимость продаж'] + sale['Себестоимость']
sale['Кол-во'] = sale['Количество продаж'] + sale['Количество-продажи']

In [23]:
sale.drop(columns= ['Количество-продажи', 'Сумма продаж с баллами', 'Себестоимость продаж', 'Сумма продаж', 'Количество продаж', 'Себестоимость'],
        inplace = True)

In [24]:
stock = stock.merge(sale, how='left', on='Код товара')

In [26]:
stock = stock.merge(ost, how='left', on='ID Сайта')

In [29]:
stock.drop_duplicates(subset=['Код товара'], inplace=True)

In [32]:
stock.drop(columns=['Код товара'], inplace=True)

In [34]:
stock.fillna(0, inplace=True)

In [35]:
sales_total = stock.groupby(by=['ID Сайта', 'Артикул поставщика', 'Товарная группа', 'Товарное направление', 'Название товара', 'Остаток'], as_index=False)[['Себес', 'Кол-во',  'Сумма']].sum()

In [36]:
sales_total

,ID Сайта,Артикул поставщика,Товарная группа,Товарное направление,Название товара,Остаток,Себес,Кол-во,Сумма
0,1000030,С0975-607,СИ СЕРЬГИ РАЗН,СИ,Серьги-пуссеты с чёрн.шпин. покрытие: платина,41.0,58433.70,56.0,1.104244e+05
1,1000181,4-008Ч,СИ КОЛЬЦО БК,СИ,Кольцо черн.,53.0,58166.10,182.0,1.303740e+05
2,1000188,13-72-0009-10091,СИ СЕРЬГИ РАЗН,СИ,Серьги с голуб. и б/ц фиан. род.,672.0,485328.71,1651.0,1.453051e+06
3,1000336,12158,СИ КОЛЬЦО ИФ,СИ,Кольцо с фиан. род.,24.0,49345.40,30.0,1.100725e+05
4,1000457,И1-1051-23-00,СИ КОЛЬЦО БК,СИ,Кольцо,1.0,0.00,0.0,0.000000e+00
...,...,...,...,...,...,...,...,...,...
51942,8380059,33-78-000M-28716,СИ ПОДАРКИ КРОСС,СИ,"Колье с крист. Swar. св. сапфир,сирен. поз., р...",36.0,36076.47,66.0,5.975105e+04
51943,8380065,33-72-000S-28716,СИ ПОДАРКИ КРОСС,СИ,"Колье с крист. Swar. б/ц род., р-р 45-50",13.0,5290.00,10.0,8.966380e+03
51944,8380085,12-72-000И-28716,СИ ПОДАРКИ КРОСС,СИ,Серьги пуссеты с крист. Swar. жёлт. металлик род.,2.0,10563.09,33.0,1.098827e+04
51945,8380087,12-72-000Ф-28716,СИ ПОДАРКИ КРОСС,СИ,Серьги пуссеты с крист. Swar. гол. металлик род.,3.0,7362.58,23.0,9.787840e+03


In [37]:
sales_total.drop_duplicates(subset=['ID Сайта'], inplace=True)

In [39]:
cb = sales_total[sales_total['Товарное направление'] == 'ЦБ']
si = sales_total[sales_total['Товарное направление'] == 'СИ']

,ID Сайта,Артикул поставщика,Товарная группа,Товарное направление,Название товара,Остаток,Себес,Кол-во,Сумма
5,1000477,47-00-0000-80119,ЦБ БРАСЛЕТЫ,ЦБ,Браслет декор.,310.0,954484.03,102.0,2.011996e+06
21,1001247,47-91-0000-28920,ЦБ БРАСЛЕТЫ,ЦБ,Браслет на нитке сердце с а/г,2909.0,5824874.85,15089.0,1.197677e+07
26,1001697,310-01-0050-30087#БЛОК,ЦБ ЦЕПИ,ЦБ,Цепь Лав с а/г,53.0,627227.08,39.0,1.310788e+06
29,1001822,332-00-0065-37677,ЦБ ЦЕПИ,ЦБ,Цепь Каприз ручной,26.0,1759976.01,19.0,3.769972e+06
76,1004490,"НБ15-031 D0,35",ЦБ БРАСЛЕТЫ,ЦБ,Браслет Косичка из 3-х цепей,61.0,246403.09,16.0,4.991027e+05
...,...,...,...,...,...,...,...,...,...
51896,8378871,41-01-0080-30200,ЦБ БРАСЛЕТЫ,ЦБ,Браслет Нонна пуст с а/г,9446.0,28087173.34,3103.0,6.389790e+07
51899,8378875,31-00-0035-30555,ЦБ ЦЕПИ,ЦБ,Цепь Якорь пуст,5726.0,363583.56,116.0,8.290518e+05
51902,8378877,31-01-0040-30205,ЦБ ЦЕПИ,ЦБ,Цепь Якорь квадрат.пуст с а/г,702.0,136492.16,42.0,3.182020e+05
51919,8379225,38-00-00M0-28668,ЦБ ЦЕПИ,ЦБ,Колье каучук,172.0,379097.31,145.0,8.517671e+05


In [41]:
cb.sort_values(by=['Сумма'], ascending=False, inplace=True)
si.sort_values(by=['Сумма'], ascending=False, inplace=True)

In [42]:
cb = cb[:100]
si = si[:100]

In [43]:
result = pd.concat([cb, si], ignore_index=True)

In [44]:
result

,ID Сайта,Артикул поставщика,Товарная группа,Товарное направление,Название товара,Остаток,Себес,Кол-во,Сумма
0,8348333,БР 57-09,ЦБ БРАСЛЕТЫ,ЦБ,Браслет Ролекс с а/г род.,3603.0,2.022024e+08,10984.0,3.746985e+08
1,1394885,331-01-0050-30176,ЦБ ЦЕПИ,ЦБ,Цепь Ромб двойной пуст с а/г,8400.0,8.632879e+07,8598.0,1.821986e+08
2,2246822,31-01-0060-30176,ЦБ ЦЕПИ,ЦБ,Цепь Ромб двойной пуст с а/г,2730.0,4.864038e+07,3536.0,1.084188e+08
3,1752241,332-0Z-0070-31025,ЦБ ЦЕПИ,ЦБ,Цепь Бисмарк с а/г ручной,13.0,5.353331e+07,1007.0,9.723379e+07
4,1047287,431-01-0060-30200,ЦБ БРАСЛЕТЫ,ЦБ,Браслет Нонна пуст с а/г,3691.0,4.530388e+07,7085.0,9.502875e+07
...,...,...,...,...,...,...,...,...,...
195,6918987,НЦ-9251037-160,СИ ЦЕПЬ,СИ,Цепь Снейк мягкий род.,1667.0,1.950348e+06,2344.0,4.739863e+06
196,1447551,422-7Р-0110-31027,СИ БРАСЛЕТ БК,СИ,Браслет ручной бисмарк с черн.,1603.0,2.244968e+06,1425.0,4.723403e+06
197,3740042,40-55040-23,СИ ЦЕПЬ,СИ,Цепь Ромб двойной с а/г род.,1539.0,1.656743e+06,3489.0,4.651702e+06
198,1902385,422-7Р-0100-31027,СИ БРАСЛЕТ БК,СИ,Браслет Бисмарк ручной черн.,1083.0,1.896998e+06,1521.0,4.642388e+06


In [45]:
result.to_excel(r'C:\Users\Trenkin.Sergey\Desktop\notebooks\archive\result.xlsx', index=False)

In [46]:
images = result['ID Сайта']

In [47]:
images.to_excel('imagestop.xlsx', index=False)

0      1863703
1      1170467
2      8348333
3      1576139
4      1245677
        ...   
195    3618947
196    3701849
197    1160491
198    4866007
199    1950133
Name: ID Сайта, Length: 200, dtype: object